In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_csv('../data/train_featureV1.csv')
test = pd.read_csv('../data/test_featureV1.csv')

In [3]:
train.head()

,uid,label,voice_opp_num_unique_count,voice_opp_num_count,voice_opp_head_unique_count,voice_opp_len_3,voice_opp_len_5,voice_opp_len_6,voice_opp_len_7,voice_opp_len_8,...,"('w_a_upflow_max', 0.0)","('w_a_upflow_max', 1.0)","('w_a_upflow_min', 0.0)","('w_a_upflow_min', 1.0)","('w_a_upflow_median', 0.0)","('w_a_upflow_median', 1.0)","('w_a_upflow_mean', 0.0)","('w_a_upflow_mean', 1.0)","('w_a_upflow_sum', 0.0)","('w_a_upflow_sum', 1.0)"
0,u0001,0,22.0,79.0,17.0,0.0,3.0,0.0,0.0,0.0,...,10720264.0,3360841.0,0.0,0.0,5563.0,9788.5,168850.068063,188654.245000,161251815.0,37730849.0
1,u0002,0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,22319.0,0.0,0.0,0.0,2453.0,0.0,4433.551724,0.000000,128573.0,0.0
2,u0003,0,15.0,21.0,10.0,0.0,1.0,0.0,0.0,2.0,...,3709153.0,8915918.0,0.0,0.0,3469.0,4784.0,37904.870672,182601.945946,37222583.0,33781360.0
3,u0004,0,77.0,254.0,31.0,0.0,1.0,0.0,0.0,12.0,...,27479806.0,7399037.0,0.0,0.0,4416.0,48402.5,129466.396501,414593.694805,177627896.0,127694858.0
4,u0005,0,55.0,401.0,28.0,0.0,4.0,0.0,0.0,0.0,...,1422573.0,12441601.0,0.0,40.0,4040.0,4595.0,38306.330341,228585.511905,21336626.0,19201183.0


In [4]:
dtrain = lgb.Dataset(train.drop(['uid','label'],axis=1),label=train.label)
dtest = lgb.Dataset(test.drop(['uid'],axis=1))

In [5]:
lgb_params =  {
    'boosting_type': 'gbdt',
    'objective': 'binary',
#    'metric': ('multi_logloss', 'multi_error'),
    #'metric_freq': 100,
    'is_training_metric': False,
    'min_data_in_leaf': 10,
    'num_leaves': 80,
    'learning_rate': 0.08,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'verbosity':-1,
    'max_depth':7,
    'colsample_bytree':0.7,
#    'gpu_device_id':2,
#    'device':'gpu'
#    'lambda_l1': 0.001,
#    'skip_drop': 0.95,
#    'max_drop' : 10
    #'lambda_l2': 0.005
    #'num_threads': 18
}    

In [6]:
def evalMetric(preds,dtrain):
    
    label = dtrain.get_label()
    
    
    pre = pd.DataFrame({'preds':preds,'label':label})
    pre= pre.sort_values(by='preds',ascending=False)
    
    auc = metrics.roc_auc_score(pre.label,pre.preds)

    pre.preds=pre.preds.map(lambda x: 1 if x>=0.38 else 0)

    f1 = metrics.f1_score(pre.label,pre.preds)
    
    
    res = 0.6*auc +0.4*f1
    
    return 'res',res,True
    

    

### 本地CV

In [7]:
lgb.cv(lgb_params,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=5,num_boost_round=10000,nfold=3,metrics=['evalMetric'])

[5]	cv_agg's res: 0.772679 + 0.0110465
[10]	cv_agg's res: 0.791244 + 0.0164082
[15]	cv_agg's res: 0.79439 + 0.0122797
[20]	cv_agg's res: 0.795218 + 0.0132017
[25]	cv_agg's res: 0.790554 + 0.0157802
[30]	cv_agg's res: 0.792676 + 0.0150801
[35]	cv_agg's res: 0.793474 + 0.0160903
[40]	cv_agg's res: 0.792307 + 0.0160219
[45]	cv_agg's res: 0.794032 + 0.0167521
[50]	cv_agg's res: 0.793758 + 0.0160414
[55]	cv_agg's res: 0.796689 + 0.0153033
[60]	cv_agg's res: 0.797657 + 0.0142221
[65]	cv_agg's res: 0.798137 + 0.0150761
[70]	cv_agg's res: 0.79684 + 0.0152859
[75]	cv_agg's res: 0.797776 + 0.0151821
[80]	cv_agg's res: 0.796405 + 0.0174218
[85]	cv_agg's res: 0.795073 + 0.0132728
[90]	cv_agg's res: 0.791653 + 0.0132963
[95]	cv_agg's res: 0.792884 + 0.0127169
[100]	cv_agg's res: 0.792877 + 0.014141
[105]	cv_agg's res: 0.790511 + 0.0133742
[110]	cv_agg's res: 0.789804 + 0.0148747
[115]	cv_agg's res: 0.788492 + 0.0150022
[120]	cv_agg's res: 0.788406 + 0.0151868
[125]	cv_agg's res: 0.786955 + 0.014198

{'res-mean': [0.6240759693177127,
  0.6467686786272657,
  0.6541183576506658,
  0.7490303257891501,
  0.7726792757071385,
  0.784029832784042,
  0.7874444912735851,
  0.790020670297141,
  0.7901484568198384,
  0.7912440294698753,
  0.7909901709243927,
  0.7957702765519433,
  0.7939516900869682,
  0.7956819165038983,
  0.7943896587450335,
  0.7929019700199212,
  0.7941176554880748,
  0.7955166337735419,
  0.7956515379592535,
  0.7952182018772119,
  0.7932076399120014,
  0.7934533256191663,
  0.7922248941859861,
  0.7923149464192684,
  0.7905536321711422,
  0.7905404276547238,
  0.7919053078120544,
  0.7913808709712118,
  0.7920031038072927,
  0.7926763506795153,
  0.7942141817477587,
  0.794973929631379,
  0.7930645976113206,
  0.7930874434694412,
  0.7934735413993107,
  0.7927306230607453,
  0.7928209866855558,
  0.7923957878617737,
  0.7925424244754292,
  0.79230743196426,
  0.7916792995611769,
  0.7924448909185654,
  0.7931469815430585,
  0.7922867729091227,
  0.7940318010149091,
  0

## 训练

In [8]:
model =lgb.train(lgb_params,dtrain,feval=evalMetric,verbose_eval=5,num_boost_round=300,valid_sets=[dtrain])

[5]	training's res: 0.834973
[10]	training's res: 0.880333
[15]	training's res: 0.90466
[20]	training's res: 0.910608
[25]	training's res: 0.923667
[30]	training's res: 0.937897
[35]	training's res: 0.944029
[40]	training's res: 0.950221
[45]	training's res: 0.956134
[50]	training's res: 0.958449
[55]	training's res: 0.961735
[60]	training's res: 0.963643
[65]	training's res: 0.967927
[70]	training's res: 0.969798
[75]	training's res: 0.970065
[80]	training's res: 0.977405
[85]	training's res: 0.97893
[90]	training's res: 0.982615
[95]	training's res: 0.98479
[100]	training's res: 0.988296
[105]	training's res: 0.989834
[110]	training's res: 0.990969
[115]	training's res: 0.994251
[120]	training's res: 0.995797
[125]	training's res: 0.997115
[130]	training's res: 0.998004
[135]	training's res: 0.998447
[140]	training's res: 0.998891
[145]	training's res: 0.998892
[150]	training's res: 0.999334
[155]	training's res: 0.999556
[160]	training's res: 0.999556
[165]	training's res: 0.999778


### 预测

In [9]:
pred=model.predict(test.drop(['uid'],axis=1))

In [10]:
res =pd.DataFrame({'uid':test.uid,'label':pred})


In [11]:
res=res.sort_values(by='label',ascending=False)
res.label=res.label.map(lambda x: 1 if x>=0.38 else 0)
#res.label = res.label.map(lambda x: int(x))

In [12]:
res.to_csv('../result/lgb-baseline.csv',index=False,header=False,sep=',',columns=['uid','label'])